In [1]:

from transformers.models.distilbert.configuration_distilbert import DistilBertConfig

from transformers import DistilBertTokenizerFast

from transformers.configuration_utils import PretrainedConfig

from model_distilbert import DistilBertForConditionalGeneration

import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

config = DistilBertConfig.from_pretrained("monologg/distilkobert")
config.type_vocab_size=1
config.layer_norm_eps= 1e-05
config.hidden_dropout_prob= 0.1
config.attention_probs_dropout_prob= 0.1
config.intermediate_size= 3072
config.hidden_act="gelu"
config.decoder_start_token_id = torch.tensor(2)
config.use_cache = True

tokenizer = DistilBertTokenizerFast.from_pretrained("monologg/distilkobert")
model = DistilBertForConditionalGeneration("monologg/distilkobert", config).to(device)

text = '근대 경제학에서는 행위의 합리성 문제를 비용 대 편익이라는 효율성 측면에서 접근하는데, 이러한 사고방식은 경제학을 넘어 사회과학 전반으로 널리 확산되 었다. 하지만 경제적 논리로는 설명하기 어렵거나 바람직하지 않은 결과를 낳은 현상도 많이 존재한다. 특히 도덕이나 규범은 경제 현상의 작동 방식에도 영향을 미치기 때문에 이를 설명하는 확장된 경제 이론이 필요하다. 도덕경제론은 바로 이러한 문제를 다루는 접근 방식이다. 이에 따르면 경제 활동에서 개인의 효용 극대화와 합리적 선택을 강조하는 경제학의 기본 가정 역시 초시대적으로 통용 되는 가치가 아니라 자본주의 등장 이후에 형성된 역사적 현상에 불과하다. 경제 적 원칙은 당대의 도덕적 규범에 의해 구성되는 상대적인 가치이기 때문에, 경제 활동이나 영역과 관련된 규범적 측면을 고려해 재구성해야 한다. ‘도덕경제 (moral economy)’론은 바로 이처럼 경제 현상에서 규범이나 문화의 역할 문제 를 다루는 접근 방식이다. 이러한 시도는 경제와 관련된 사회 현상을 이해하는 데 에서 경제학적 접근 방식의 편협성과 한계를 해결하는 데에도 풍부한 시사점을 줄수있을것이다. 이논문은기존의도덕경제론에서다룬주요쟁점과개념을 소개하고, 이 논의가 미디어 경제를 이해하는 데 주는 함의, 쟁점 등을 검토한다.'

input = tokenizer(text, return_tensors='pt')
label = tokenizer('천하통일', return_tensors='pt')

output = model(**input.to(device), labels=label['input_ids'].to(device))

/opt/conda/envs/final/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at monologg/distilkobert were not used when initializing DistilBertModel: ['distilbert.transformer.layer.2.attention.v_lin.bias', 'distilbert.transformer.layer.2.attention.q_lin.weight', 'distilbert.transformer.layer.1.attention.q_lin.weight', 'distilbert.transformer.layer.1.sa_layer_norm.weight', 'distilbert.transformer.layer.2.attention.v_lin.weight', 'distilbert.transformer.layer.1.attention.k_lin.weight', 'distilbert.transformer.layer.1.attention.v_lin.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.2.output_layer_norm.we

In [3]:
output

Seq2SeqLMOutput(loss=tensor(8.7542, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[[ 0.6114, -0.3501,  1.4667,  ...,  0.2065, -0.3554,  0.7851],
         [-0.0768,  0.5385,  0.9510,  ...,  0.5405, -1.0165,  0.5869],
         [ 0.7476, -0.5816,  0.6950,  ...,  0.4870, -1.3124,  0.6337]]],
       device='cuda:0', grad_fn=<AddBackward0>), past_key_values=((tensor([[[[ 0.4898,  0.2701, -0.2470,  ...,  1.1429, -0.2002, -0.3766],
          [ 1.0107,  0.3930, -0.7696,  ...,  1.5234, -0.1480, -0.4280],
          [ 0.8531,  0.3423, -0.1208,  ..., -0.0831,  0.8597, -0.3816]],

         [[-0.2985,  0.8396,  0.4669,  ...,  1.1254,  0.5441,  0.9839],
          [-0.1441,  1.1577,  0.1470,  ...,  0.9710, -0.4099,  0.9341],
          [-0.5297,  1.7554,  0.1474,  ...,  0.6767, -0.0782,  0.4501]],

         [[ 0.0322, -0.8545,  0.4244,  ..., -0.7264, -0.2966, -0.0507],
          [ 0.0937, -0.7003,  0.3404,  ..., -0.9574,  0.0372, -0.2609],
          [-0.1677, -0.4239,  0.3417,  ..., -0.15

In [ ]:
 summary_ids = model.generate(torch.tensor([input_ids]), num_beams=num_beams, **generation_args.__dict__)
        print('** text: ', text)
        print('** title: ', title)
        if len(summary_ids.shape) == 1  or summary_ids.shape[0] == 1:
            ## 출력 1개
            title = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
            print('Gen title 0', title)
        else :
            ## 출력 여러개
            titles = tokenizer.batch_decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
            for idx, title in enumerate(titles) :
                print('Gen title', idx, title)